In [6]:
# input txt file
ClozeTest = '109choices.txt'
ArticleTest = '109_article.txt'

In [2]:
! pip install -U sentence-transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelWithLMHead, pipeline
from sentence_transformers import SentenceTransformer, util
import torch

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (1.1.0)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking")
model_cloze = AutoModelWithLMHead.from_pretrained("bert-large-uncased-whole-word-masking")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeque

In [4]:
# read the logic of the article
def Read(text):
    q = False
    a = False
    article = []
    question = []
    answer = []
    ques_each_arcticle = []

    for i in range(len(text)):
        if text[i][0:7]=='Article':
            ques_each_arcticle.append(0)
            text[i] = text[i].replace(('\n'), ' ').replace(('\t'), '')
            article.append(text[i])
            q = False
            a = False
        elif text[i][0] == 'Q' and str.isdigit(text[i][1]):
            if len(ques_each_arcticle) != 0:
              ques_each_arcticle[-1]+=1
            text[i] = text[i].replace(('\n'), ' ').replace(('\t'), '')
            question.append(text[i])
            q = True
            a = False
        elif text[i][0] == 'A' and str.isdigit(text[i][1]):
            text[i] = text[i].replace(('\n'), ' ').replace(('\t'), '')
            answer.append(text[i])
            q = False
            a = True
        elif q is True and not(text[i][0] == 'Q' and str.isdigit(text[i][1])):
            text[i] = text[i].replace(('\n'), ' ').replace(('\t'), '')
            question.append(text[i])
            question[-2 :] = [''.join(question[-2 :])]
        elif a is True and not(text[i][0] == 'A' and str.isdigit(text[i][1])):
            text[i] = text[i].replace(('\n'), ' ').replace(('\t'), '')
            answer.append(text[i])
            answer[-2 :] = [''.join(answer[-2 :])]
        else:
            text[i] = text[i].replace(('\n'), ' ').replace(('\t'), '')
            article.append(text[i])
            article[-2 :] = [''.join(article[-2 :])]
    return article, question, answer, ques_each_arcticle

In [7]:
f = open(ClozeTest, 'r')
text = f.readlines()
q = False
a = False
question = []
answer = []
# transform the text to list
_, question, answer, _= Read(text)
f.close()

In [15]:
import numpy as np

def input_txt(Q,i, model):
  f = Q[i]
  F = f.replace('$', '{}')
  sequence = F.format(tokenizer.mask_token)
  input = tokenizer.encode(sequence, return_tensors="pt")
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
  token_logits = model(input)[0]
  mask_token_logits = token_logits[0, mask_token_index, :]
  top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
  return top_5_tokens

def choice(A, Num):
  f = A[Num]
  label = ['(A)', '(B)', '(C)', '(D)']
  array = []
  for j in range(3):
    array.append(A[Num][f.find(label[j]) + 4:f.find(label[j+1]) - 1])
  array.append(A[Num][f.find(label[3])+4:])
  return array


def Similarity_cloze(model, top_five, answer_option):
  voc1 = answer_option
  voc2 = top_five
  embeddings1 = model.encode(voc1, convert_to_tensor=True)
  embeddings2 = model.encode(voc2, convert_to_tensor=True)
  answer_item = ['A', 'B', 'C', 'D']
  cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
  for i in range(len(voc1)):
    for j in range(len(voc2)):
      print("{} \t\t {} \t\t Score: {:.4f}".format(voc1[i], voc2[j], cosine_scores[i][j]))
    print('\n')
  # for i in range(len(voc1)):
  #   print('(' + answer_item[i] + ')' + voc1[i] + ':', np.sum(cosine_scores.numpy(), axis=1)[i])
  return cosine_scores.numpy()

def Answering_ordering(cos_score, answer_option):
  score = []
  rank = []
  order = []
  voc1 = answer_option
  for i in range(len(voc1)):
    score.append(np.max(cos_score, axis=1)[i])
  answer_item = ['A', 'B', 'C', 'D']
  rank = np.argsort(score)[::-1]
  for i in range (len(voc1)):
    order.append(answer_item[rank[i]])
  return order

In [16]:
answer_list=[]

for i in range(len(question)):
  Answer_option = choice(answer, i)
  Similarity_score = Similarity_cloze(SentenceTransformer('stsb-mpnet-base-v2'), tokenizer.decode(input_txt(question,i,model_cloze)).split(' '), Answer_option)
  # print('sentence: ',question[i].replace('$', '____'))
  answer_order = Answering_ordering(Similarity_score, Answer_option)
  answer_list.append((question[i][1:3].replace('.',''), answer_order))
# print(answer_list)
with open(ClozeTest.replace('.txt', '_output') + '.txt', 'w') as f:
  for i in range(len(answer_list)):
    f.write('A'+answer_list[i][0]+'.')
    for j in range(len(answer_list[i][1])):
      f.write('('+answer_list[i][1][j]+')')
      if(j!=len(answer_list[i][1])-1):
        f.write('->')
    f.write('\n')
# top_k_accuracy=0
# k=1
# cloze_answer_108 = ['A','A','D','C','D','B','B','C','B','D','A','C','D','D']
# cloze_answer_109 = ['A','C','A','A','C','B','C','D','D','A','A','D','D']
# print(len(answer_list), len(cloze_answer_109))
# for i in range(len(answer_list)):
#   for j in range(k):
#     if answer_list[i][1][j] == cloze_answer_109[i]:
#       top_k_accuracy+=1
# print(top_k_accuracy)

gloves 		 heat 		 Score: 0.2900
gloves 		 gloves 		 Score: 1.0000
gloves 		 towel 		 Score: 0.4229
gloves 		 hose 		 Score: 0.3219
gloves 		 sponge 		 Score: 0.2364


jacket 		 heat 		 Score: 0.2796
jacket 		 gloves 		 Score: 0.4324
jacket 		 towel 		 Score: 0.3004
jacket 		 hose 		 Score: 0.1906
jacket 		 sponge 		 Score: 0.2568


ring 		 heat 		 Score: 0.2515
ring 		 gloves 		 Score: 0.2931
ring 		 towel 		 Score: 0.2026
ring 		 hose 		 Score: 0.2477
ring 		 sponge 		 Score: 0.2958


socks   		 heat 		 Score: 0.1611
socks   		 gloves 		 Score: 0.3216
socks   		 towel 		 Score: 0.2105
socks   		 hose 		 Score: 0.2848
socks   		 sponge 		 Score: 0.3385


check 		 lift 		 Score: 0.2510
check 		 get 		 Score: 0.4898
check 		 raise 		 Score: 0.3327
check 		 take 		 Score: 0.4284
check 		 move 		 Score: 0.2235


kick 		 lift 		 Score: 0.2308
kick 		 get 		 Score: 0.3904
kick 		 raise 		 Score: 0.3759
kick 		 take 		 Score: 0.3221
kick 		 move 		 Score: 0.4026


raise 		 lift 		 Score: 0.43

Article:Method 2

In [18]:
model_name = "bert-large-cased-whole-word-masking-finetuned-squad"
qa_model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer_article = AutoTokenizer.from_pretrained(model_name)
#model to calculate cosine distance
model_sentence = SentenceTransformer('stsb-mpnet-base-v2')

In [19]:
def Similarity_Article(model_sentence, answer_from_bert, Answer_option):
  voc1 = answer_from_bert
  voc2 = Answer_option
  embeddings1 = model_sentence.encode(voc1, convert_to_tensor=True)
  embeddings2 = model_sentence.encode(voc2, convert_to_tensor=True)
  answer_item = ['A', 'B', 'C', 'D']
  cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2).flatten()
  for i in range(len(voc2)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(voc1, voc2[i], cosine_scores[i]))
  best_answer = torch.argmax(cosine_scores)
  print('(' + answer_item[best_answer.item()] + ')' + voc2[best_answer])
  return cosine_scores.numpy(), answer_item[best_answer.item()]

def Answering_ordering_article(cos_score, answer_option):
  rank = []
  order = []
  voc1 = answer_option
  answer_item = ['A', 'B', 'C', 'D']
  rank = np.argsort(cos_score)[::-1]
  for i in range (len(voc1)):
    order.append(answer_item[rank[i]])
  print('Output order: ',order)
  return order

In [20]:
# read article as input
f = open(ArticleTest, 'r')
text = f.readlines()
q = False
a = False
article = []
question = []
answer = []
ques_each_arcticle = []
article, question, answer, ques_each_arcticle = Read(text)
f.close()
text = article

In [21]:
# summarizer = pipeline("summarization")
answer_article_list=[]
count = 0
for num_article, amount in enumerate(ques_each_arcticle) :
  for i in range(amount):
    question[count+i]
    inputs = tokenizer_article(question[count+i], text[num_article], add_special_tokens=True, return_tensors="pt")
    inputs_ids = inputs["input_ids"].tolist()[0]
    outputs = qa_model(**inputs)
    # print(summarizer(text[num_article], max_length=135, min_length=30)[0]['summary_text'])
    Answer_option = choice(answer, count+i)
    # answer from bert
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer_from_bert = tokenizer_article.convert_tokens_to_string(tokenizer_article.convert_ids_to_tokens(inputs_ids[answer_start:answer_end]))
    print(f"Question: {question[count+i]}")
    print(f"Answer: {answer_from_bert}")
    similarity_score_article, answer_item = Similarity_Article(model_sentence, answer_from_bert, Answer_option)
    answer_order = Answering_ordering_article(similarity_score_article, Answer_option)
    answer_article_list.append((question[count+i][1:3].replace('.',''), answer_order))
  count += amount

with open(ArticleTest.replace('.txt', '_output') + '.txt', 'w') as f:
  for i in range(len(answer_article_list)):
    f.write('A'+answer_article_list[i][0]+'.')
    for j in range(len(answer_article_list[i][1])):
      f.write('('+answer_article_list[i][1][j]+')')
      if(j!=len(answer_article_list[i][1])-1):
        f.write('->')
    f.write('\n')

# top_k_accuracy=0
# k=1
# article_answer_108 = ['D','B','A','A','B','B','A','C','D','B','D','D','C','C','D']
# article_answer_109 = ['C','C','A','D','D','B','B','B','D','A','C','B','D','D','C','D','B','C']

# for i in range(len(answer_article_list)):
#   for j in range(k):
#     if answer_article_list[i][1][j] == article_answer_109[i]:
#       top_k_accuracy+=1
# print(top_k_accuracy)

Question: Q16. What is "it" in the reading? 
Answer: 
 		 The sun. 		 Score: -0.0362
 		 The rain. 		 Score: 0.0721
 		 The wind. 		 Score: 0.0898
 		 The rainbow.   		 Score: 0.1175
(D)The rainbow.  
Output order:  ['D', 'C', 'B', 'A']
Question: Q17. How does Dad feel“when it comes”? 
Answer: Dad ’ s face falls
Dad ’ s face falls 		 He is scared. 		 Score: 0.1860
Dad ’ s face falls 		 He is excited. 		 Score: 0.1107
Dad ’ s face falls 		 He is not happy. 		 Score: 0.2534
Dad ’ s face falls 		 He is not interested.    		 Score: 0.1478
(C)He is not happy.
Output order:  ['C', 'A', 'D', 'B']
Question: Q18. What can we learn about Buffy’s friends? 
Answer: I made some good friends at my last yard sale.
I made some good friends at my last yard sale. 		 Matt has experience of yard sales. 		 Score: 0.4553
I made some good friends at my last yard sale. 		 Jamie had a successful yard sale last week. 		 Score: 0.4210
I made some good friends at my last yard sale. 		 Albert thinks weather is imp